In [1]:
import requests
from abc import ABC, abstractmethod
from typing import List, Optional
import importlib
import backend.adapters.repository as repository
import backend.domain.model as model
import backend.service_layer.service as service
import backend.service_layer.uow as uow
from sec_api import XbrlApi
import pandas as pd
import os
from backend.domain.model import Company, Filing
from backend.service_layer.uow import SqlAlchemyUnitOfWork
from analyze_company_data_completeness import get_sp500_tickers

c:\Users\Site Oracle\reduceComplexity\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

dfs = pd.read_html(r"C:\Users\Site Oracle\reduceComplexity\Map from Fundamental Accounting Concepts to XBRL Taxonomy Concepts.html")

In [21]:
dfs[0].to_csv('test.csv')

In [29]:

unique_filers = service.find_unique_companies_with_recent_10q_filings(os.getenv("SEC_API_KEY"))

In [30]:
unique_filers[:10]

[{'cik': '1084869',
  'ticker': 'FLWS',
  'company_name': '1 800 FLOWERS COM INC',
  'filing_date': '2025-05-09T09:59:51-04:00'},
 {'cik': '1770787',
  'ticker': 'TXG',
  'company_name': '10x Genomics, Inc.',
  'filing_date': '2025-05-08T17:35:37-04:00'},
 {'cik': '1877461',
  'ticker': 'CBDW',
  'company_name': '1606 CORP.',
  'filing_date': '2025-05-13T16:32:56-04:00'},
 {'cik': '1690080',
  'ticker': 'ATNF',
  'company_name': '180 Life Sciences Corp.',
  'filing_date': '2025-05-15T16:25:55-04:00'},
 {'cik': '1599407',
  'ticker': 'EFSH',
  'company_name': '1847 Holdings LLC',
  'filing_date': '2025-05-15T16:23:14-04:00'},
 {'cik': '34782',
  'ticker': 'SRCE',
  'company_name': '1ST SOURCE CORP',
  'filing_date': '2025-04-24T16:01:15-04:00'},
 {'cik': '38723',
  'ticker': '',
  'company_name': '1st FRANKLIN FINANCIAL CORP',
  'filing_date': '2025-05-20T16:48:18-04:00'},
 {'cik': '1600641',
  'ticker': 'DIBS',
  'company_name': '1stdibs.com, Inc.',
  'filing_date': '2025-05-12T08:34:2

In [39]:
sp500 = get_sp500_tickers()

In [10]:
uow_instance = uow.SqlAlchemyUnitOfWork()

In [40]:
for ticker in sp500[:100]:
    print(ticker)
    try:
        import sys
        import os
        
        # Redirect stdout and stderr to devnull
        original_stdout = sys.stdout
        original_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, 'w')
        
        try:
            res = service.get_consolidated_income_statements(ticker, uow_instance, "10-K", use_database=True)
            res.df.to_csv(f'{ticker}_test.csv')
        finally:
            # Restore stdout and stderr
            sys.stdout.close()
            sys.stderr.close()
            sys.stdout = original_stdout
            sys.stderr = original_stderr
        print(res.analyze_period_coverage()['has_continuous_coverage'])

    except Exception as e:
        print(f'{ticker} failed')
        print(e)


MMM
True
AOS
True
ABT
True
ABBV
True
ACN
True
ADBE
True
AMD
True
AES
True
AFL
True
A
True
APD
True
ABNB
True
AKAM
True
ALB
True
ARE
True
ALGN
True
ALLE
True
LNT
True
ALL
True
GOOGL
True
GOOG
True
MO
True
AMZN
True
AMCR
True
AEE
True
AEP
True
AXP
True
AIG
True
AMT
True
AWK
True
AMP
True
AME
True
AMGN
True
APH
True
ADI
True
ANSS
True
AON
True
APA
True
APO
True
AAPL
True
AMAT
True
APTV
True
ACGL
True
ADM
True
ANET
True
AJG
True
AIZ
True
T
True
ATO
True
ADSK
True
ADP
True
AZO
True
AVB
AVB failed
argument of type 'NoneType' is not iterable
AVY
True
AXON
True
BKR
True
BALL
True
BAC
True
BAX
True
BDX
True
BRK.B
BRK.B failed
No CIK found for ticker: BRK.B
BBY
True
TECH
True
BIIB
True
BLK
True
BX
True
BK
True
BA
True
BKNG
True
BSX
True
BMY
True
AVGO
True
BR
True
BRO
True
BF.B
BF.B failed
No CIK found for ticker: BF.B
BLDR
True
BG
True
BXP
True
CHRW
True
CDNS
True
CZR
True
CPT
True
CPB
True
COF
True
CAH
True
KMX
True
CCL
True
CARR
True
CAT
True
CBOE
True
CBRE
True
CDW
True
COR
True
CNC
True
CNP


In [44]:
pm_index = next((i for i, filer in enumerate(unique_filers) if filer['ticker'] == 'PM'), None)
print(f"PM found at index: {pm_index}")
if pm_index is not None:
    print(f"PM data: {unique_filers[pm_index]}")


PM found at index: 3916
PM data: {'cik': '1413329', 'ticker': 'PM', 'company_name': 'Philip Morris International Inc.', 'filing_date': '2025-04-24T08:26:13-04:00'}


In [46]:
for filer in unique_filers[3916:]:
    ticker = filer['ticker']
    print(ticker)
    try:
        import sys
        import os
        
        # Redirect stdout and stderr to devnull
        original_stdout = sys.stdout
        original_stderr = sys.stderr
        sys.stdout = open(os.devnull, 'w')
        sys.stderr = open(os.devnull, 'w')
        
        try:
            res = service.get_consolidated_income_statements(ticker, uow_instance, "10-K", use_database=True)
            # res.df.to_csv(f'{ticker}_test.csv')
        finally:
            # Restore stdout and stderr
            sys.stdout.close()
            sys.stderr.close()
            sys.stdout = original_stdout
            sys.stderr = original_stderr
        print(res.analyze_period_coverage()['has_continuous_coverage'])

    except Exception as e:
        print(f'{ticker} failed')
        print(e)


PM
True

 failed
No CIK found for ticker: 
PSX
True
PECO
True
PHIO
True

 failed
No CIK found for ticker: 
PXPC
True
PHR
True
PHUN
True
PLL
True
PDM
True
PAPLF
True
PBFS
True
PLNH
True
PLNT
True
PLAG
True
PL
True
AGS
True
PLYA
True
PLTK
True
PLMK
False
PLMJF
True
PLUR
True
PLYM
True
POLA
True
PII
True
PMHS
True
PDLB
True
PNYG
True
PRCH
True
PTLO
True
PTMN
True
POST
True
PSTL
PSTL failed
Incompatible indexer with Series
PW
True
PROP
True
PRAX
True
PRPO
True
POAI
True
PRLD
True
PINC
True
PRMLF
True
SQFT
True
PRME
True
PRI
True
FRST
True
PRMB
True
PRIM
True
BPRN
True

 failed
No CIK found for ticker: 
PRTH
True
PRVA
True
ACDC
True
PMN
True
PUMP
True
PRPH
True
AGQ
True
PCSA
True
IPDN
True
PAL
True
PROF
True
PGNY
True
PLD
True

 failed
No CIK found for ticker: 
PPCB
True

 failed
No CIK found for ticker: 

 failed
No CIK found for ticker: 
PTIX
True
PTGX
True
PLX
True
TARA
True
PRLB
True
PKTX
True

 failed
No CIK found for ticker: 
PVBC
True
PUBM
True
PSA
True
PULM
True
LUNG
True
PSTG
PSTG 

In [36]:
test1 = service.get_consolidated_income_statements('ATNF', uow_instance, "10-K", use_database=False)


      [MAP] No tags found for 'Revenue' in this statement.
      [MAP] No tags found for 'COGS' in this statement.
      [MAP] No tags found for 'CapitalExpenditures' in this statement.
Years covered in last filing (2025-03-31):
Index(['2024-01-01:2024-12-31', '2023-01-01:2023-12-31'], dtype='object', name='date_range')
Index(['2024-01-01:2024-12-31', '2023-01-01:2023-12-31'], dtype='object', name='date_range')
Index(['2024-01-01:2024-12-31', '2023-01-01:2023-12-31'], dtype='object', name='date_range')
loading filings for [<backend.domain.model.Filing object at 0x00000291E3387B50>, <backend.domain.model.Filing object at 0x00000291E3387110>, <backend.domain.model.Filing object at 0x00000291E3386650>, <backend.domain.model.Filing object at 0x00000291E33864D0>, <backend.domain.model.Filing object at 0x00000291E33874D0>]
no cover page for 0001213900-25-026273 10-K
loading filings for 2022-12-31 10-K
loading filings for 2020-12-31 10-K
loading filings for 2018-12-31 10-K
loading filings for

In [37]:
test1.financial_statements[0].table

Index(['2024-01-01:2024-12-31', '2023-01-01:2023-12-31'], dtype='object', name='date_range')


date_range,2024-01-01:2024-12-31,2023-01-01:2023-12-31
metric,,
LossOnAssetImpairment,1526542.00,9063000.00
ChangeInFairValueOfPreferredStockLiability,1577305.00,NaN
ResearchAndDevelopmentExpense,1401435.00,2303751.00
OtherResearchAndDevelopmentExpense,567832.00,480777.00
GeneralAndAdministrativeExpense,4355462.00,10646417.00
OtherSellingGeneralAndAdministrativeExpense,NaN,46555.00
OperatingExpenses,6324729.00,13477500.00
OperatingIncomeLoss,-6324729.00,-13477500.00
OtherIncome,2822864.00,21074.00


In [26]:
res.financial_statements[2].table

Index(['2017-01-01:2017-12-31', '2016-01-01:2016-12-31',
       '2018-01-01:2018-12-31'],
      dtype='object', name='date_range')


date_range,2017-01-01:2017-12-31,2016-01-01:2016-12-31,2018-01-01:2018-12-31
metric,,,
Revenue,1.817640e+10,1.716880e+10,2.037760e+10
COGS,2.112000e+09,1.705000e+09,2.943000e+09
NoninterestIncomeOtherOperatingIncome,1.457000e+09,1.678000e+09,1.360000e+09
InterestAndFeeIncomeLoansAndLeases,8.148000e+09,7.214000e+09,9.941000e+09
InterestAndDividendIncomeSecurities,8.900000e+07,1.310000e+08,1.180000e+08
InterestIncomeDepositsWithFinancialInstitutions,3.260000e+08,1.390000e+08,5.470000e+08
InterestExpenseDeposits,7.790000e+08,5.980000e+08,1.287000e+09
InterestExpenseLongTermDebt,1.333000e+09,1.107000e+09,1.656000e+09
CardMemberServices,1.392000e+09,1.100000e+09,1.777000e+09


In [ ]:
import pandas as pd

df = pd.read_csv('ABT_test.csv')
df

In [ ]:
res = service.get_consolidated_income_statements("GES", uow_instance, "10-Q", use_database=False)
res.df 

In [ ]:
consolidated = service.get_consolidated_income_statements("GD", uow_instance, "10-K")

In [ ]:
consolidated.df

In [ ]:


with SqlAlchemyUnitOfWork() as uow:
    # Get initial statements
    statements = service.get_consolidated_income_statements("GOOG", uow, "10-K")
    
    uow.stmts.add(statements)

# Start new UoW to verify changes persisted
with SqlAlchemyUnitOfWork() as uow:
    saved_statements = uow.stmts.get("GOOG", "10-K")
    assert saved_statements is not None

In [ ]:
saved_statements.df

In [ ]:
from backend.adapters.repository import PostgresCombinedFinancialStatementsRepository
with SqlAlchemyUnitOfWork() as uow:
    pcfs = PostgresCombinedFinancialStatementsRepository(uow)
    print(pcfs._serialize(statements))

In [ ]:
uow_instance = uow.UnitOfWork()


In [ ]:
combined = service.get_consolidated_income_statements('B ABA', uow_instance, '10-Q')

In [ ]:
for filing in combined.source_filings:
    print(filing.cover_page.document_fiscal_period_focus, filing.cover_page.document_fiscal_year_focus) 

In [5]:
def remove_zero_decimals(val):
    try:
        num = float(val)
        if num == int(num):
            return int(num)
        else:
            return num
    except (ValueError, TypeError):
        return val

In [6]:
import pandas as pd
pd.options.display.float_format = '{:.10g}'.format
combined.df = combined.df.map(remove_zero_decimals)

In [ ]:
combined.df

In [ ]:
combined.df.apply(combined.is_sparse_row, axis=1)

In [9]:
def convert_to_millions(val):
    try:
        num = float(val)
        
        if abs(num) >= 100_000:
            result = round(num / 1000000, 2)
            # Check if the result has no meaningful decimal part
            if result == int(result):
                return int(result)
            else:
                return result
        else:
            return num
    except (ValueError, TypeError):
        return val

In [10]:
def is_sparse_row(row):
    # Count zero values in the row
    numeric_values = 0
    zero_values = 0
    
    for val in row:
        try:
            num = float(val)
            numeric_values += 1
            if num == 0:
                zero_values += 1
        except (ValueError, TypeError):
            # Skip non-numeric values
            pass
    
    # If no numeric values, don't delete
    if numeric_values == 0:
        return False
        
    # Calculate percentage of zeros
    zero_percentage = zero_values / numeric_values if numeric_values > 0 else 0
    
    # Return True if more than 50% zeros
    return zero_percentage > 0.5

In [11]:
# combined.df = combined.df.applymap(convert_to_millions)


In [12]:
# combined.df = combined.df.loc[[not is_sparse_row(row) for _, row in combined.df.iterrows()]]

In [ ]:
combined.df

In [ ]:
combined.analyze_period_coverage()

In [ ]:
df

In [ ]:
for filing in combined.source_filings:
    try: 
        print(filing.cover_page.document_fiscal_period_focus, filing.cover_page.document_fiscal_year_focus)
    except:
        print(filing.form)

In [ ]:
sorted_columns = sorted(combined.df.columns, key=lambda x: x.split(':')[0])
print(sorted_columns)

In [ ]:
# Get data for first 4 filings
for filing in company.filings[:1]:
    filing.data = uow_instance.sec_filings.get_filing_data(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )
    filing.filing_url = uow_instance.sec_filings.get_filing_url(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')


In [ ]:
res = company.select_filings_with_processing_pattern(company.filings,'10-Q')

In [ ]:
for filing in res:
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')